# Libraries

##  Remove warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Built-in libraries

In [ ]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import time
import pandas    as pd
import numpy     as np
from   tqdm      import tqdm

# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Visualization library
#
import matplotlib.pyplot   as plt 


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Sklearn library
#
from sklearn.preprocessing import StandardScaler


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Tensorflow library
#
from   tensorflow.keras.models                  import *
from   tensorflow.keras.layers                  import *
from   tensorflow.keras.callbacks               import *
from   tensorflow.keras.metrics                 import *
from   tensorflow.keras.optimizers              import *
from   tensorflow.keras.utils                   import plot_model


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User library
#
from   utils.Evaluation                          import RegressionEvaluation
from   utils.LossFunctions                       import *
from   utils.CyclicLR                            import *
from   utils.WarmUpCosineDecayScheduler          import *
from   utils.Attention                           import *

# Parameters

## General parameters

In [ ]:
# Parameters
#
filename   = 'Data/Austin_Weather.csv'

# Dictionary with the performance of each model
#
Performance = {}

## Neural networks parameters

In [ ]:
Lag        = 24
Horizon    =  3

epochs     = 100
batch_size =  64

## Neural network metrics/loss/optimizer

In [ ]:
# Define NN-metrics
#
metrics = [ MeanAbsolutePercentageError(name="MAPE", dtype=None),
            RootMeanSquaredError(name='RMSE', dtype=None) ]

# Define Loss function
#
loss = SMAPE

# Define Opitmizer
#
optimizer = Adam(learning_rate = 1e-3)

## Neural Network callbacks

In [ ]:
# Earlystopping
#
earlystopping = EarlyStopping(monitor       = 'val_loss', 
                              min_delta     = 0,
                              patience      = 30,
                              verbose       = 1, 
                              mode          = 'min', 
                              #
                              restore_best_weights = True)
  
    
# Reduce LR on Plateau
#
lrs_scheduler   = ReduceLROnPlateau(monitor   = 'val_loss', 
                                   factor     = 0.5,
                                   patience   = 5)

# Cyclic LR
#
CyclicLR_scheduler = CyclicLR(base_lr   = 0.001, 
                              max_lr    = 0.006, 
                              step_size = 2000., 
                              mode      = 'triangular')
# Cosine Decay
#
CosLR__scheduler = WarmUpCosineDecayScheduler(learning_rate_base=0.001)



# Define callbacks
#
callbacks = [earlystopping, CyclicLR_scheduler]

# Data handling

## Import data


In [ ]:
# Start timer
#
start = time.time()

# Load data
#
df = pd.read_csv( filename )

print('[INFO] Data imported')
print('[INFO] Time: %.2f seconds' % (time.time() - start))

## Preprocess data

### Set index

In [ ]:
# Convert Date to 'datetime64'
#
df['Date'] = df['Date'].astype('datetime64')

# Set index
#
df.set_index('Date', inplace=True)

df.head(3)

### Create new features based on Date

In [ ]:
# Get features
#
Features = ['Temperature [Fahrenheit]', 'DewPoint [Fahrenheit]']

# df['DayOfWeek'] = df.index.dayofweek
# df['Month']     = df.index.month

### Split Training/Testing

In [ ]:
idx = int( df.shape[0] * 0.9 )

df_train = df[ :idx ]
df_test  = df[ idx: ]

### Visualization

In [ ]:
for feature in Features:
    
    plt.figure( figsize=(20, 2) )
    
    df_train[feature].plot( color='tab:blue' )
    df_test[feature].plot( color='tab:orange')
    
    plt.legend(['Training', 'Testing'], frameon = False)
    plt.ylabel(feature)
    plt.title('Feature: {}'.format(feature))

### Fixing Lag

In [ ]:
df_test = pd.concat([df_train.iloc[-Lag:], df_test])

### Scaling

In [ ]:
# Setup scaler
#
scaler = StandardScaler()

df_train = pd.DataFrame( scaler.fit_transform(df_train), 
                         index   = df_train.index,
                         columns = df_train.columns )


df_test = pd.DataFrame( scaler.transform(df_test), 
                        index    = df_test.index,
                        columns  = df_test.columns )

## Create Training/Testing data

In [ ]:
def create_dataset(df = None, Lag = 1, Horizon = 12, ForecastingSeries = []):
    
    if (ForecastingSeries == []):
        ForecastingSeries = df.columns
    
    dataX, dataY = [], []
    for i in tqdm( range(df.shape[0] + 1  - Lag - Horizon) ):
        
        dataX.append( df.to_numpy()[i:(i+Lag)] )        
        dataY.append( df[ ForecastingSeries ].to_numpy()[i + Lag : i + Lag + Horizon] )
        
        
    return ( np.array(dataX), np.array(dataY) )

In [ ]:
trainX, trainY = create_dataset(df = df_train, 
                                Lag = Lag, 
                                Horizon = Horizon, 
                                ForecastingSeries = Features)

testX,  testY  = create_dataset(df = df_test, 
                                Lag = Lag, 
                                Horizon = Horizon, 
                                ForecastingSeries = Features)

print('Training instances:   %6i' % trainX.shape[0])
print('Testing instances:    %6i' % testX.shape[0])

# Seq2Seq LSTM

## Setup model

In [ ]:
model = Sequential()

model.add(LSTM(100, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(RepeatVector( Horizon ))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(trainY.shape[2], activation='linear')))


model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training process

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/LSTM-AE.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['Seq2Seq LSTM'] = Performance_Foresting_Model

# CNN-LSTM 

## Setup model

In [ ]:
Input_layer = Input(shape=(trainX.shape[1], trainX.shape[2])) 

conv = Conv1D(filters =  8, kernel_size = 3, activation = 'relu')(Input_layer)
conv = Conv1D(filters = 16, kernel_size = 7, activation = 'relu')(conv)

lstm = LSTM(100, return_sequences=True, activation='relu')(conv)
dropout = Dropout(0.2)(lstm)
lstm = LSTM(100, activation='relu')(dropout)

dense = Dense(Horizon*trainY.shape[2], activation='relu')(lstm)

Output_layer = Reshape((Horizon, trainY.shape[2]))(dense)

model = Model([Input_layer], [Output_layer])



model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training performance

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training process performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/CNN-LSTM.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['CNN-LSTM'] = Performance_Foresting_Model

# TCN

## Setup model

In [ ]:
from TCN.tcn import *

Input_layer = Input(shape=( trainX.shape[1], trainX.shape[2]) )   

x = TCN(nb_filters=32, kernel_size=2, padding='same', activation='relu')(Input_layer)
#
x = Flatten()(x)
#
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Output layer
x = Dense(Horizon*trainY.shape[2], activation='relu')(x)
Output_layer = Reshape((Horizon, trainY.shape[2]))(x)

model = Model([Input_layer], [Output_layer])



model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training performance

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training process performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/TCN.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['TCN'] = Performance_Foresting_Model

# Seq2Seq CNN-LSTM 

## Setup model

In [ ]:
model = Sequential()

model.add( Conv1D(filters = 4, kernel_size = 3, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])) )
model.add( Conv1D(filters = 8, kernel_size = 7, activation='relu') )
model.add( MaxPooling1D(pool_size=2) )
#
model.add(Flatten())
#
model.add( RepeatVector( Horizon ) )
model.add( LSTM(200, activation='relu', return_sequences=True) )
#
model.add(TimeDistributed( Dense(100, activation='relu')) )
model.add(TimeDistributed( Dense(trainY.shape[2], activation='linear')) )



model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training process

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training process performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/Seq2Seq-CNN-LSTM.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['Seq2Seq CNN-LSTM'] = Performance_Foresting_Model

# Multi-Head CNN-LSTM Model

## Setup model

In [ ]:
Input_layer = Input(shape=(trainX.shape[1], trainX.shape[2])) 

head_list = []
for i in range(0, trainX.shape[2]):
    Conv_layer_head = Conv1D(filters=4, kernel_size=7,  activation='relu')(Input_layer)
    Conv_layer_head = Conv1D(filters=6, kernel_size=11, activation='relu')(Conv_layer_head)
    Conv_layer_flatten = Flatten()(Conv_layer_head)
    head_list.append( Conv_layer_flatten )
    
    
Concat_cnn = Concatenate(axis=1)(head_list)
reshape = Reshape((head_list[0].shape[1], trainX.shape[2]))(Concat_cnn)

lstm = LSTM(100, activation='relu')(reshape)
repeat = RepeatVector( Horizon )(lstm)
lstm_2 = LSTM(100, activation='relu', return_sequences=True)(repeat)
dropout = Dropout(0.2)(lstm_2)
output_layer = Dense(trainY.shape[2], activation='linear')(dropout)

model = Model(inputs=Input_layer, outputs=output_layer)

    

model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training process

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training process performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/MultiHead-CNN-LSTM.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['Multi-head CNN-LSTM'] = Performance_Foresting_Model

# CNN-Att Model

## Setup model

In [ ]:
Input_layer = Input(shape=( trainX.shape[1], trainX.shape[2]) )   


x = Conv1D(filters=32, kernel_size=2, padding='same', activation='relu' )(Input_layer)
x = MaxPool1D(2)(x)
x = Conv1D(filters=64, kernel_size=2, padding='same', activation='relu' )(x)
x = Attention(100)(x)
#
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Output layer
x = Dense(Horizon*trainY.shape[2], activation='relu')(x)
Output_layer = Reshape((Horizon, trainY.shape[2]))(x)

model = Model([Input_layer], [Output_layer])



model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training process

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training process performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/CNN-Att.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['CNN-Att'] = Performance_Foresting_Model

# LSTM-Att Model

## Setup model

In [ ]:
# https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb


# Inputs
#
Inputs = Input(shape=(trainX.shape[1], trainX.shape[2]))


# Encoder
#
encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(units = 32, 
                                                       activation='relu', 
#                                                        dropout=0.2, 
#                                                        recurrent_dropout=0.2, 
                                                       return_sequences=True, 
                                                       return_state=True)(Inputs)

encoder_last_h = BatchNormalization(momentum=0.6)(encoder_last_h)
encoder_last_c = BatchNormalization(momentum=0.6)(encoder_last_c)



# Decoder
#
decoder_input = RepeatVector( trainY.shape[1] )(encoder_last_h)

decoder_stack_h = LSTM(units = 32, 
                       activation='relu', 
#                        dropout=0.2, 
#                        recurrent_dropout=0.2,
                       return_state=False, 
                       return_sequences=True)(decoder_input, initial_state=[encoder_last_h, encoder_last_c])


# Attention
#
attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
attention = Activation('softmax')(attention)

context = dot([attention, encoder_stack_h], axes=[2,1])
context = BatchNormalization(momentum=0.6)(context)

# Merging
#
decoder_combined_context = concatenate([context, decoder_stack_h])

# Output
#
# out = Dense( Horizon, activation='linear' )(decoder)
Outputs = TimeDistributed(Dense(trainY.shape[2], activation='linear'))(decoder_combined_context)

model = Model(inputs = Inputs, outputs = Outputs)



model.compile(loss      = loss, 
              optimizer = optimizer, 
              metrics   = metrics)

plot_model(model = model, rankdir = "ΤΒ", show_shapes = True)

## Training process

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = batch_size, 
                  callbacks        = callbacks,
                  verbose          = 1, 
                  validation_split = 0.1)


# Terminate clock
#
print('[INFO] Time = %.2f' % (time.time() - start))

## Training process performance

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 4))

ax[0].plot( score.history['RMSE']     );
ax[0].plot( score.history['val_RMSE'] );
ax[0].legend(['Training', 'Validation'], frameon=False);
ax[0].set_xlabel('Epochs');
ax[0].set_ylabel('RMSE');

ax[1].plot( score.history['MAPE']     );
ax[1].plot( score.history['val_MAPE'] );
ax[1].legend(['Training', 'Validation'], frameon=False);
ax[1].set_xlabel('Epochs');
ax[1].set_ylabel('MAPE');

## Save forecasting model

In [ ]:
# Save model
#
model.save('models/LSTM-Att.hdf5')

## Evaluation

### Get predictions

In [ ]:
y_pred = model.predict( testX )

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
for idx, feature in enumerate(Features):
    
    print('[INFO] Feature: ', feature)
    print('------------------------------------------------')
    Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

    for i in range( Horizon ):
    
        Prices = pd.DataFrame([])        

        Prices['Real']       = testY[:,  i, idx]
        Prices['Prediction'] = y_pred[:, i, idx]

        
        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )
        
        # Store results
        #
        Performance_Foresting_Model['RMSE']    += [ RMSE    ]
        Performance_Foresting_Model['MAE']     += [ MAE     ]
        Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
        Performance_Foresting_Model['R2']      += [ R2      ]
        
        # Present results
        #
        print('Horizon: ', i)
        print('> RMSE:  ', RMSE)
        print('> SMAPE: ', SMAPE)
        print('> R2:    ', R2)
        print()
        
Performance['LSTM-Att'] = Performance_Foresting_Model

# Performance evaluation

In [ ]:
def PlotPerformance(Metric='RMSE', Performance=None):
    
    nMethods = len(Performance) 
    
    fig, ax = plt.subplots(nrows = 1, ncols = nMethods, figsize=(20, 4))
    for i, method in enumerate( Performance ):

        ax[i].bar(x = np.arange(Horizon)+1, height = Performance[ method ][ Metric ], width = .5 )
        ax[i].legend([method], frameon=False)

        ax[i].set_ylim([0, np.max(Performance[ method ][ Metric ]) * 1.2])


    fig.suptitle('Metric: {}'.format(Metric), size=14);    
    fig.text(0.5, -0.1, 'Forecasting horizon', ha='center', size=14);
    fig.text(0.08, 0.5, Metric, va='center', rotation='vertical', size=14);

In [ ]:
PlotPerformance('RMSE', Performance)

In [ ]:
PlotPerformance('SMAPE', Performance)

In [ ]:
PlotPerformance('R2', Performance)